<a href="https://colab.research.google.com/github/jorgelinaju/Agustin/blob/main/ia_integraciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =======================================================================
# PASO 1: INSTALACIÓN Y CONFIGURACIÓN
# =======================================================================

# 1.1 - Instalar la librería de Google para IA Generativa
!pip install -q -U google-generativeai

In [ ]:
# 1.2 - Importar las librerías necesarias
import google.generativeai as genai
import json
from google.colab import userdata # Para manejar la API Key de forma segura

# 1.3 - Configurar la API Key
# NOTA PARA EL ALUMNO: Antes de ejecutar, guarda tu API Key en el gestor de "Secrets"
# de Colab con el nombre 'GOOGLE_API_KEY'. Esto es más seguro que pegarla aquí.

try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("✅ API Key configurada correctamente.")
except Exception as e:
    print("🚨 Error al configurar la API Key. Asegúrate de guardarla en los Secrets de Colab.")
    # Puedes agregar aquí un input() para pegarla manualmente como alternativa
    # GOOGLE_API_KEY = input("Pega tu API Key aquí: ")
    # genai.configure(api_key=GOOGLE_API_KEY)

🚨 Error al configurar la API Key. Asegúrate de guardarla en los Secrets de Colab.


In [ ]:
# 1.4 - Inicializar el modelo que vamos a utilizar
# Usaremos 'gemini-1.5-flash' por ser rápido y eficiente.
model = genai.GenerativeModel('gemini-1.5-flash')
print(f"🤖 Modelo '{model.model_name}' inicializado.")

🤖 Modelo 'models/gemini-1.5-flash' inicializado.


In [ ]:
# =======================================================================
# PASO 2: LA FUNCIÓN UNIFICADA
# =======================================================================

def generar_paquete_de_contenido(tema: str, descripcion_persona: str) -> dict:
    """
    Genera un paquete completo de contenido sobre un tema, adoptando una personalidad específica.

    Esta función toma un tema y la descripción de una persona, construye un prompt
    detallado para un LLM, ejecuta la llamada a la API y parsea la respuesta JSON.

    Args:
        tema: El tema sobre el que se creará el contenido (ej: "La computación cuántica").
        descripcion_persona: Un párrafo detallado describiendo la personalidad,
                             tono y estilo que la IA debe adoptar.

    Returns:
        Un diccionario de Python con el paquete de contenido estructurado,
        o un diccionario de error si falla la generación o el parseo.
    """
    # Aquí es donde ocurre la verdadera magia: el diseño del prompt.
    # Usamos un f-string para insertar las variables directamente en la plantilla.
    prompt_template = f"""
    **Tu Rol y Tarea Principal:**
    Actúa rigurosamente como la siguiente persona:
    ---
    {descripcion_persona}
    ---
    Tu tarea es generar un paquete de contenido multimedia sobre el siguiente tema:
    **Tema:** {tema}

    **Reglas de Formato de Salida (MUY IMPORTANTE):**
    1.  Tu respuesta debe ser **EXCLUSIVAMENTE** un objeto JSON válido, sin ningún texto, explicación o markdown antes o después.
    2.  El JSON debe seguir estrictamente la siguiente estructura y claves:
        {{
          "tema_solicitado": "...",
          "persona_utilizada": "...",
          "contenido_generado": {{
            "titulo_atractivo": "...",
            "resumen_ejecutivo": "...",
            "post_para_linkedin": "...",
            "pregunta_para_debate": "...",
            "analogia_o_metafora": "..."
          }}
        }}

    **Instrucciones Adicionales:**
    -   Asegúrate de que cada pieza de contenido refleje fielmente la personalidad descrita.
    -   No inventes datos, pero sé creativo dentro de los límites del rol asignado.
    -   La clave "persona_utilizada" debe ser un resumen muy breve de la persona que te pasé.

    **COMIENZA LA SALIDA JSON A CONTINUACIÓN:**
    """

    try:
        # Hacemos la llamada al modelo
        response = model.generate_content(prompt_template)

        # Limpiamos la respuesta para asegurar que es un JSON válido
        # A veces, el modelo puede devolver el JSON dentro de un bloque de código markdown
        clean_response_text = response.text.replace("```json", "").replace("```", "").strip()

        # Parseamos el string JSON a un diccionario de Python
        contenido_generado = json.loads(clean_response_text)
        return contenido_generado

    except json.JSONDecodeError:
        # Error común: la IA no devolvió un JSON perfectamente formateado
        return {"error": "No se pudo decodificar la respuesta JSON.", "respuesta_recibida": response.text}
    except Exception as e:
        # Otro tipo de error en la llamada a la API
        return {"error": f"Ocurrió un error inesperado: {e}"}

In [ ]:
# =======================================================================
# PASO 3: EJEMPLOS DE PERSONAS Y EJECUCIÓN
# =======================================================================

# 3.1 - Definimos el tema a tratar (será el mismo para la comparación)
tema_a_tratar = "La semana laboral de 4 días"

# 3.2 - Definimos las TARJETAS DE PERSONA (cada alumno tendrá una diferente)

persona_analista_senior = """
Eres un analista de negocios senior para una consultora internacional como McKinsey.
Tu tono es profesional, formal, escéptico y extremadamente basado en datos.
Usas terminología de negocios (KPIs, ROI, overhead, bottom-line).
Siempre presentas las dos caras de la moneda (pros y contras), pero con un
enfoque en los riesgos financieros y la eficiencia operativa.
Tu objetivo es informar a directivos (CEOs, CFOs) para que tomen decisiones calculadas.
No usas emojis.
"""

persona_coach_bienestar = """
Eres una coach de mindfulness y bienestar laboral. Tu nombre es "Luna".
Tu tono es empático, inspirador y positivo. Usas un lenguaje cercano y emocional.
Hablas de "balance vida-trabajo", "burnout", "bienestar del equipo" y "energía renovada".
Te enfocas exclusivamente en los beneficios para la salud mental y la felicidad de los empleados.
Tu objetivo es convencer a managers de RRHH y empleados de que la felicidad impulsa la productividad.
Usas emojis cálidos y positivos (🧘‍♀️✨💖🌿).
"""

# 3.3 - Ejecutamos el motor para cada persona
print(f"🚀 Generando contenido para el tema: '{tema_a_tratar}'...")
print("-" * 50)

print("👤 Ejecutando con la Persona: Analista Senior...")
paquete_analista = generar_paquete_de_contenido(tema_a_tratar, persona_analista_senior)
print("✅ ¡Contenido de Analista generado!")

print("\n👤 Ejecutando con la Persona: Coach de Bienestar...")
paquete_coach = generar_paquete_de_contenido(tema_a_tratar, persona_coach_bienestar)
print("✅ ¡Contenido de Coach generado!")

# =======================================================================
# PASO 4: COMPARACIÓN DE RESULTADOS
# =======================================================================

print("\n\n" + "="*50)
print("📊 RESULTADOS COMPARATIVOS 📊")
print("="*50)

print("\n--- TÍTULO ATRACTIVO ---")
print(f"  Analista: {paquete_analista.get('contenido_generado', {}).get('titulo_atractivo', 'N/A')}")
print(f"  Coach:    {paquete_coach.get('contenido_generado', {}).get('titulo_atractivo', 'N/A')}")

print("\n--- POST PARA LINKEDIN ---")
print(f"  Analista:\n    {paquete_analista.get('contenido_generado', {}).get('post_para_linkedin', 'N/A').replace('\n', '\n    ')}")
print(f"\n  Coach:\n    {paquete_coach.get('contenido_generado', {}).get('post_para_linkedin', 'N/A').replace('\n', '\n    ')}")

print("\n--- PREGUNTA PARA DEBATE ---")
print(f"  Analista: {paquete_analista.get('contenido_generado', {}).get('pregunta_para_debate', 'N/A')}")
print(f"  Coach:    {paquete_coach.get('contenido_generado', {}).get('pregunta_para_debate', 'N/A')}")


# Opcional: Guardar los resultados en archivos JSON
with open('paquete_analista.json', 'w', encoding='utf-8') as f:
    json.dump(paquete_analista, f, ensure_ascii=False, indent=4)

with open('paquete_coach.json', 'w', encoding='utf-8') as f:
    json.dump(paquete_coach, f, ensure_ascii=False, indent=4)

print("\n\n💾 Archivos 'paquete_analista.json' y 'paquete_coach.json' guardados.")

🚀 Generando contenido para el tema: 'La semana laboral de 4 días'...
--------------------------------------------------
👤 Ejecutando con la Persona: Analista Senior...
✅ ¡Contenido de Analista generado!

👤 Ejecutando con la Persona: Coach de Bienestar...
✅ ¡Contenido de Coach generado!


📊 RESULTADOS COMPARATIVOS 📊

--- TÍTULO ATRACTIVO ---
  Analista: Semana laboral de 4 días: ¿Oportunidad de Mejora o Riesgo Financiero?
  Coach:    ¡Descubre el secreto para equipos felices y productivos: La semana de 4 días! ✨💖

--- POST PARA LINKEDIN ---
  Analista:
    La semana laboral de 4 días: ¿innovación o ilusión?  Análisis recientes muestran resultados dispares en cuanto a la productividad.  Mientras algunos reportan mejoras en el bienestar del empleado, la evidencia de un aumento significativo del ROI es inconsistente.  Un análisis profundo del impacto en los costos laborales, la eficiencia operativa y la posible disminución de la productividad durante los 4 días es crucial antes de adoptar 

In [ ]:
# =======================================================================
# VISUALIZADOR PARA UN ARCHIVO DENTRO DE COLAB
# =======================================================================

import ipywidgets as widgets
from IPython.display import display, HTML
import json
import os # Usaremos 'os' para verificar si el archivo existe

def visualizar_un_archivo(nombre_del_archivo):
    """
    Carga un único archivo JSON, lo formatea en HTML y lo muestra de forma
    atractiva directamente en una celda de Colab.

    Args:
        nombre_del_archivo (str): La ruta al archivo JSON que se quiere visualizar.
    """
    # Primero, verificamos que el archivo realmente exista
    if not os.path.exists(nombre_del_archivo):
        print(f"🚨 Error: El archivo '{nombre_del_archivo}' no fue encontrado.")
        return

    print(f"📄 Visualizando el contenido de: {nombre_del_archivo}")

    try:
        # Abrimos y cargamos el contenido del archivo JSON
        with open(nombre_del_archivo, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Extraemos la información de forma segura
        persona = data.get('persona_utilizada', 'Persona Desconocida')
        tema = data.get('tema_solicitado', 'N/A')
        contenido = data.get('contenido_generado', {})

        # --- Creamos el contenido HTML para la visualización ---
        # puede mejorarse:
        html_content = f"""
          <div
            style="
              max-width: 800px;
              margin: 0 auto;
              background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
              border-radius: 20px;
              padding: 2px;
              box-shadow: 0 20px 40px rgba(0, 0, 0, 0.1);
              font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            "
          >
            <div
              style="
                background: white;
                border-radius: 18px;
                padding: 40px;
                position: relative;
                overflow: hidden;
              "
            >
              <!-- Header con gradiente -->
              <div
                style="
                  background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                  margin: -40px -40px 30px -40px;
                  padding: 30px 40px;
                  border-radius: 18px 18px 0 0;
                  position: relative;
                "
              >
                <div
                  style="
                    background: rgba(255, 255, 255, 0.1);
                    border-radius: 50px;
                    padding: 15px 25px;
                    display: inline-block;
                    margin-bottom: 15px;
                    backdrop-filter: blur(10px);
                  "
                >
                  <span style="font-size: 24px">👤</span>
                  <span
                    style="
                      color: white;
                      font-weight: 600;
                      margin-left: 10px;
                      font-size: 16px;
                    "
                  >
                    PERFIL PROFESIONAL
                  </span>
                </div>
                <h1
                  style="
                    color: white;
                    margin: 0;
                    font-size: 28px;
                    font-weight: 700;
                    line-height: 1.3;
                    text-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
                  "
                >
                  {persona}
                </h1>
              </div>

              <!-- Tema destacado -->
              <div
                style="
                  background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);
                  padding: 20px 25px;
                  border-radius: 15px;
                  margin-bottom: 30px;
                  position: relative;
                  overflow: hidden;
                "
              >
                <div
                  style="
                    position: absolute;
                    top: -50%;
                    right: -20px;
                    width: 100px;
                    height: 100px;
                    background: rgba(255, 255, 255, 0.1);
                    border-radius: 50%;
                  "
                ></div>
                <h3
                  style="
                    color: white;
                    margin: 0 0 8px 0;
                    font-size: 14px;
                    font-weight: 600;
                    text-transform: uppercase;
                    letter-spacing: 1px;
                  "
                >
                  TEMA PRINCIPAL
                </h3>
                <p
                  style="
                    color: white;
                    margin: 0;
                    font-size: 18px;
                    font-weight: 500;
                    position: relative;
                    z-index: 1;
                  "
                >
                  {tema}
                </p>
              </div>

              <!-- Grid de contenido -->
              <div style="display: grid; gap: 25px">
                <!-- Título Atractivo -->
                <div
                  style="
                    background: #f8fafc;
                    border: 1px solid #e2e8f0;
                    border-radius: 12px;
                    padding: 25px;
                    position: relative;
                    transition: all 0.3s ease;
                  "
                >
                  <div
                    style="
                      background: #3b82f6;
                      color: white;
                      padding: 8px 15px;
                      border-radius: 20px;
                      font-size: 12px;
                      font-weight: 600;
                      text-transform: uppercase;
                      letter-spacing: 0.5px;
                      display: inline-block;
                      margin-bottom: 15px;
                    "
                  >
                    ✨ Título Atractivo
                  </div>
                  <h3
                    style="
                      color: #1e293b;
                      margin: 0;
                      font-size: 22px;
                      font-weight: 700;
                      line-height: 1.4;
                    "
                  >
                    {contenido.get('titulo_atractivo', 'N/A')}
                  </h3>
                </div>

                <!-- Post para LinkedIn -->
                <div
                  style="
                    background: #f8fafc;
                    border: 1px solid #e2e8f0;
                    border-radius: 12px;
                    padding: 25px;
                    position: relative;
                  "
                >
                  <div
                    style="
                      background: #0077b5;
                      color: white;
                      padding: 8px 15px;
                      border-radius: 20px;
                      font-size: 12px;
                      font-weight: 600;
                      text-transform: uppercase;
                      letter-spacing: 0.5px;
                      display: inline-block;
                      margin-bottom: 15px;
                    "
                  >
                    💼 Post para LinkedIn
                  </div>
                  <div
                    style="
                      background: white;
                      border-left: 4px solid #0077b5;
                      padding: 20px;
                      border-radius: 8px;
                      box-shadow: 0 2px 10px rgba(0, 0, 0, 0.05);
                    "
                  >
                    <p
                      style="
                        color: #475569;
                        margin: 0;
                        font-size: 16px;
                        line-height: 1.6;
                        font-style: italic;
                      "
                    >
                      {contenido.get('post_para_linkedin', 'N/A').replace('\n', '<br />')}
                    </p>
                  </div>
                </div>

                <!-- Pregunta para Debate -->
                <div
                  style="
                    background: linear-gradient(135deg, #ffecd2 0%, #fcb69f 100%);
                    border-radius: 12px;
                    padding: 25px;
                    position: relative;
                    overflow: hidden;
                  "
                >
                  <div
                    style="
                      position: absolute;
                      top: -30px;
                      right: -30px;
                      width: 80px;
                      height: 80px;
                      background: rgba(255, 255, 255, 0.2);
                      border-radius: 50%;
                    "
                  ></div>
                  <div
                    style="
                      background: rgba(255, 255, 255, 0.8);
                      color: #dc2626;
                      padding: 8px 15px;
                      border-radius: 20px;
                      font-size: 12px;
                      font-weight: 600;
                      text-transform: uppercase;
                      letter-spacing: 0.5px;
                      display: inline-block;
                      margin-bottom: 15px;
                    "
                  >
                    ❓ Pregunta para Debate
                  </div>
                  <p
                    style="
                      color: #7c2d12;
                      margin: 0;
                      font-size: 18px;
                      font-weight: 600;
                      position: relative;
                      z-index: 1;
                    "
                  >
                    {contenido.get('pregunta_para_debate', 'N/A')}
                  </p>
                </div>

                <!-- Analogía o Metáfora -->
                <div
                  style="
                    background: linear-gradient(135deg, #a8edea 0%, #fed6e3 100%);
                    border-radius: 12px;
                    padding: 25px;
                    position: relative;
                    overflow: hidden;
                  "
                >
                  <div
                    style="
                      position: absolute;
                      bottom: -40px;
                      left: -40px;
                      width: 120px;
                      height: 120px;
                      background: rgba(255, 255, 255, 0.2);
                      border-radius: 50%;
                    "
                  ></div>
                  <div
                    style="
                      background: rgba(255, 255, 255, 0.8);
                      color: #7c3aed;
                      padding: 8px 15px;
                      border-radius: 20px;
                      font-size: 12px;
                      font-weight: 600;
                      text-transform: uppercase;
                      letter-spacing: 0.5px;
                      display: inline-block;
                      margin-bottom: 15px;
                    "
                  >
                    🎭 Analogía o Metáfora
                  </div>
                  <blockquote style="margin: 0; position: relative; z-index: 1">
                    <p
                      style="
                        color: #581c87;
                        margin: 0;
                        font-size: 18px;
                        font-style: italic;
                        font-weight: 500;
                        line-height: 1.5;
                        position: relative;
                      "
                    >
                      <span
                        style="
                          font-size: 40px;
                          color: rgba(88, 28, 135, 0.3);
                          position: absolute;
                          left: -10px;
                          top: -5px;
                          font-family: serif;
                        "
                        >"</span
                      >
                      <span style="margin-left: 30px"
                        >{contenido.get('analogia_o_metafora', 'N/A')}</span
                      >
                      <span
                        style="
                          font-size: 40px;
                          color: rgba(88, 28, 135, 0.3);
                          font-family: serif;
                        "
                        >"</span
                      >
                    </p>
                  </blockquote>
                </div>
              </div>

              <!-- Footer decorativo -->
              <div
                style="
                  margin-top: 30px;
                  padding-top: 20px;
                  border-top: 1px solid #e2e8f0;
                  text-align: center;
                "
              >
                <div
                  style="
                    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    width: 60px;
                    height: 4px;
                    border-radius: 2px;
                    margin: 0 auto;
                  "
                ></div>
              </div>
            </div>
          </div>
          """

        # Mostramos el HTML directamente en la celda
        display(HTML(html_content))

    except json.JSONDecodeError:
        print(f"🚨 Error: El archivo '{nombre_del_archivo}' no parece ser un JSON válido.")
    except Exception as e:
        print(f"🚨 Ocurrió un error inesperado al procesar el archivo: {e}")

In [ ]:
# --- EJEMPLO DE USO ---

# Supongamos que acabas de generar 'paquete_analista.json' y 'paquete_coach.json'

# Visualizamos el primer archivo
archivo_a_mostrar_1 = 'paquete_analista.json'
visualizar_un_archivo(archivo_a_mostrar_1)

📄 Visualizando el contenido de: paquete_analista.json
